<div style=" background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px 0; color:#fff; text-align: center">Data Processing
</h2>
</div>

In [1]:
from datetime import datetime
import pyarrow as py
import pandas as pd
import numpy as np
import time
from pytz import timezone
from datetime import datetime
import matplotlib.pyplot as plt


In [2]:
customers_processd = pd.read_csv("../Source/customers.csv", sep=";")
products_processd = pd.read_csv("../Source/products.csv", sep=";")
transactions_processd_old = pd.read_csv("../Source/Transactions.csv", sep=";") #table d'origine
transactions_processd = pd.read_csv("../Source/Data_processed/timestamps_processed.csv", sep=";") #nouvelle table processed

In [3]:
transactions_processd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     687534 non-null  object
 1   Timestamp   687534 non-null  object
 2   session_id  687534 non-null  object
 3   client_id   687534 non-null  object
dtypes: object(4)
memory usage: 21.0+ MB


In [4]:
#Etape 1 : Je merge les deux table en "outer" et intégrant un indicateur de joiture
merge_transaction_customer = pd.merge(transactions_processd, customers_processd, how='outer', on='client_id', indicator=True)
merge_transaction_customer

,id_prod,Timestamp,session_id,client_id,sex,birth,_merge
0,0_1470,2021-06-11 21:02:39.382765,s_47346,c_1,m,1955,both
1,0_513,2021-07-21 22:41:38.769525,s_65433,c_1,m,1955,both
2,0_1186,2021-07-25 12:17:34.446678,s_66947,c_1,m,1955,both
3,0_1448,2021-07-26 17:37:29.438136,s_67467,c_1,m,1955,both
4,0_1475,2021-07-27 10:30:00.293075,s_67769,c_1,m,1955,both
...,...,...,...,...,...,...,...
687550,0_1360,2023-01-15 17:53:15.077669,s_327362,c_999,m,1964,both
687551,1_351,2023-01-21 17:16:06.807620,s_330231,c_999,m,1964,both
687552,1_727,2023-01-21 17:40:50.856434,s_330231,c_999,m,1964,both
687553,0_1442,2023-01-28 13:12:42.429052,s_333631,c_999,m,1964,both


In [5]:
#Je crée un df de ma table merged pour ne récupérer que les lignes qui n'ont pas été jointes
no_data_customer = merge_transaction_customer.copy()
no_data_customer = no_data_customer.loc[no_data_customer['_merge'] != 'both']
no_data_customer

,id_prod,Timestamp,session_id,client_id,sex,birth,_merge
18399,NaN,NaN,NaN,c_1223,m,1963,right_only
167262,NaN,NaN,NaN,c_2706,f,1967,right_only
191551,NaN,NaN,NaN,c_3017,f,1992,right_only
225851,NaN,NaN,NaN,c_3443,m,1959,right_only
240172,NaN,NaN,NaN,c_3526,m,1956,right_only
261001,NaN,NaN,NaN,c_3789,f,1997,right_only
282271,NaN,NaN,NaN,c_4086,f,1992,right_only
305952,NaN,NaN,NaN,c_4358,m,1999,right_only
309172,NaN,NaN,NaN,c_4406,f,1998,right_only
312791,NaN,NaN,NaN,c_4447,m,1956,right_only


#### Il y a des client_id dans la table des customers qu'il n'y a pas dans la table transaction :

##### ---> Hypothèse 1 : Tout les clients de la base de données n'ont pas forcément acheter sur le site en ligne

In [6]:
#Etape 2 : Je merge les deux table en "outer" et intégrant un indicateur de joiture
merge_transaction_product = pd.merge(transactions_processd, products_processd, how='outer', on='id_prod', indicator=True)
merge_transaction_product

,id_prod,Timestamp,session_id,client_id,price,categ,_merge
0,0_0,2021-03-01 13:45:51.575117,s_282,c_5152,3.75,0,both
1,0_0,2021-03-02 06:42:55.351333,s_621,c_2917,3.75,0,both
2,0_0,2021-03-02 18:49:49.651862,s_852,c_3988,3.75,0,both
3,0_0,2021-03-02 21:57:33.862118,s_908,c_1004,3.75,0,both
4,0_0,2021-03-03 23:59:48.997483,s_1379,c_278,3.75,0,both
...,...,...,...,...,...,...,...
687550,2_99,2022-05-21 01:53:46.967570,s_211661,c_5828,84.99,2,both
687551,2_99,2022-07-21 01:53:46.967570,s_241374,c_5828,84.99,2,both
687552,2_99,2022-11-11 20:56:49.820935,s_296127,c_7135,84.99,2,both
687553,2_99,2022-12-11 20:56:49.820935,s_310606,c_7135,84.99,2,both


In [7]:
#Je crée un df de ma table merged pour ne récupérer que les lignes qui n'ont pas été jointes
no_data_product = merge_transaction_product.copy()
no_data_product = no_data_product.loc[no_data_product['_merge'] != 'both']
no_data_product

,id_prod,Timestamp,session_id,client_id,price,categ,_merge
6073,0_1014,NaN,NaN,NaN,1.15,0,right_only
6511,0_1016,NaN,NaN,NaN,35.06,0,right_only
8723,0_1025,NaN,NaN,NaN,24.99,0,right_only
21687,0_1062,NaN,NaN,NaN,20.08,0,right_only
40971,0_1119,NaN,NaN,NaN,2.99,0,right_only
94245,0_1318,NaN,NaN,NaN,20.92,0,right_only
297077,0_1620,NaN,NaN,NaN,0.80,0,right_only
299185,0_1624,NaN,NaN,NaN,24.50,0,right_only
311123,0_1645,NaN,NaN,NaN,2.99,0,right_only
323013,0_1780,NaN,NaN,NaN,1.67,0,right_only


#### Il y a des product_id dans la table des produits qu'il n'y a pas dans la table transaction :

##### ---> Hypothèse 2: Tout les articles de la base de données ne sont pas forcément disponible à la vente en ligne
##### ---> Hypothèse 3: Les clients de Lapage qui n'ont pas acheter sur le site internet ont peut-être acheter hors-ligne car non disponible en ligne

#### Piste de réfléxion possible sur la base des éléments observé :

##### ---> Référencer l'ensemble des produits sur le site en ligne pour voir si les clients de Lapage n'ayant jamais fait de transaction en ligne commencent à acheter en ligne 

##### C'est une analyse parallèle à mener, en attendant je vais les supprimer, car je ne peux même pas relier les produits avec les clients



In [8]:
#Merge de toute les tables avec transactions_processd
df_all = transactions_processd.merge(customers_processd, how='left', on='client_id').merge(products_processd, how='left', on='id_prod', indicator = True)
df_all = df_all.reset_index(drop=True)
df_all

,id_prod,Timestamp,session_id,client_id,sex,birth,price,categ,_merge
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,f,1967,11.99,0,both
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,m,1960,19.37,0,both
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,m,1988,4.50,0,both
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,f,1989,6.55,0,both
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,f,1956,16.49,0,both
...,...,...,...,...,...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573,f,1996,21.92,1,both
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50,f,1994,48.99,2,both
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488,f,1985,26.99,1,both
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848,m,1953,8.99,0,both


In [9]:
df_all = df_all.rename(columns={"_merge": 'merge'})
df_all.drop(columns="merge", inplace=True)
df_all['categ'] = df_all['categ'].astype(str)

In [10]:
df_all

,id_prod,Timestamp,session_id,client_id,sex,birth,price,categ
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,f,1967,11.99,0
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,m,1960,19.37,0
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,m,1988,4.50,0
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,f,1989,6.55,0
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,f,1956,16.49,0
...,...,...,...,...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573,f,1996,21.92,1
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50,f,1994,48.99,2
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488,f,1985,26.99,1
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848,m,1953,8.99,0


In [11]:
df_all['age'] = df_all['birth'].apply(lambda x: 2023 - int(x))

In [13]:
#Créer une colonne de trange d'âge
bins = [0, 18, 25, 35, 45, 55, 65, 100]
labels = ['0-18', '19-25', '26-35', '36-45', '46-55', '56-65', '66+']
df_all['age_group'] = pd.cut(df_all['age'], bins=bins, labels=labels, right=False)

In [14]:
# Joiture ok -> extraction csv du fichier processed
df_all.to_csv('../Source/Data_processed/lapage_all.csv', sep=';', index=False)

<div style="background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px; color:#fff; "> Données finale complète  </h2>
<h3 style="margin: auto; padding: 10px; color:#fff; "> --> Début du data modeling (cf fichier "data_modeling.ipynb") </h3>
</div>